# Titanic Survival Prediction - Model Development

This notebook covers Part A of the project: Data loading, preprocessing, model training (Random Forest), evaluation, and saving the model.

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# 1. Acquire & Load Data
# If running on Colab, you might need to upload the file or download it directly.
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
print(f"Data loaded. Shape: {df.shape}")
df.head()

In [ ]:
# 2. Feature Selection & Preprocessing
# Selected features: Pclass, Sex, Age, Fare, Embarked
selected_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'

X = df[selected_features]
y = df[target]

# Define Preprocessing Pipelines
numeric_features = ['Age', 'Fare', 'Pclass']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Handle missing values
    ('scaler', StandardScaler())                   # Scaling
])

categorical_features = ['Sex', 'Embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# 3. Model Training (Random Forest)
# Create Pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
print("Training model...")
clf.fit(X_train, y_train)
print("Training complete.")

In [ ]:
# 4. Evaluation
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# 5. Save Model
model_filename = 'titanic_survival_model.joblib'
joblib.dump(clf, model_filename)
print(f"Model saved to {model_filename}")

# For Colab: Code to download the file to your local machine
try:
    from google.colab import files
    files.download(model_filename)
except ImportError:
    print("Not running on Google Colab, file saved locally.")

In [ ]:
# 6. Reload Verification
loaded_clf = joblib.load(model_filename)
sample_input = X_test.iloc[:5]
print("Original Preds:", clf.predict(sample_input))
print("Loaded Preds:  ", loaded_clf.predict(sample_input))